In [2]:
import stanza
from collections import Counter 
from nltk.util import ngrams
from nltk.corpus import stopwords


In [3]:
#load russian language model for stanza
stanza.download('ru')

2020-11-08 13:09:24 INFO: Downloading default packages for language: ru (Russian)...
2020-11-08 13:09:27 INFO: File exists: /home/philipp/stanza_resources/ru/default.zip.
2020-11-08 13:09:34 INFO: Finished downloading models and saved to /home/philipp/stanza_resources.


In [4]:
#create stanza pipeline for tokenization, pos labels and lemmatization
nlp = stanza.Pipeline(lang="ru", processors='tokenize,pos,lemma')

2020-11-08 13:09:34 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |

2020-11-08 13:09:35 INFO: Use device: gpu
2020-11-08 13:09:35 INFO: Loading: tokenize
2020-11-08 13:09:39 INFO: Loading: pos
2020-11-08 13:09:41 INFO: Loading: lemma
2020-11-08 13:09:41 INFO: Done loading processors!


In [5]:
# load the text
infile_name = "hp1-1"
doc = nlp(open(f"/home/philipp/projects/text-analysis/data/{infile_name}.txt").read())

In [39]:
n = 3
top_ngrams = 1000
lemmatize = False
remove_stopwords = False

# remove all numerals, punctuation and proper nouns from list 
# extract lemmatized word for nouns, adjectives and verbs
out_str = ""
filter_pos = ["NUM", "PUNCT", "PROPN"]
lemmatize_pos = [] if not lemmatize else ["NOUN", "ADJ", "VERB", "DET", "PRON"]
additional_punctuation = "«»"
lemmas = [word for sent in doc.sentences for word in sent.words if word.upos not in filter_pos]
total_word_count = len(lemmas)
print(f"{total_word_count}\ttotal words (without {filter_pos}) \n")
lemmas = [word.lemma if word.upos in lemmatize_pos else word.text for word in lemmas]
lemmas = [lemma.lower() for lemma in lemmas if lemma not in additional_punctuation]

# get ngrams and filter out individual words
russian_stopwords = stopwords.words("russian") if remove_stopwords else []

bi_grams = list(ngrams(lemmas, n)) 
counter = Counter(bi_grams)
wc = counter.most_common(top_ngrams)

cum = 0
i=0

header = "rank\ttotal\tcum\tword\n" if n == 1 else "rank\ttotal\tword\n"
out_str+=header
for ngram, freq in [ngram for ngram in wc]:
    cum += freq/total_word_count
    i+=1
    if not any([True for i in range(len(ngram)) if ngram[i] in russian_stopwords]):
        words = " ".join(ngram)
        row = f"{i}\t{freq}\t{cum*100:.2f}%\t{words}" if n == 1 else f"{i}\t{freq}\t{words}"
        out_str+=row+"\n"
print(out_str)

76587	total words (without ['NUM', 'PUNCT', 'PROPN']) 

rank	total	word
1	47	о том что
2	32	для того чтобы
3	27	к тому же
4	24	на самом деле
5	20	в том что
6	19	судя по всему
7	18	что у него
8	17	так и не
9	16	в жизни не
10	16	никак не мог
11	14	с тех пор
12	14	все в порядке
13	13	в этот момент
14	13	у него был
15	12	выдавил из себя
16	12	до тех пор
17	12	тех пор пока
18	11	не знал что
19	10	себе под нос
20	10	никогда не видел
21	10	по крайней мере
22	10	на то что
23	10	то что он
24	9	не мог понять
25	9	на следующее утро
26	9	в тот момент
27	9	и тут же
28	9	за то что
29	9	если по правде
30	9	он никак не
31	8	он не мог
32	8	тем не менее
33	8	не сомневался что
34	8	тех пор как
35	8	потому что он
36	8	прежде чем он
37	8	в конце концов
38	8	тот момент когда
39	8	никогда в жизни
40	8	и с трудом
41	8	ему показалось что
42	8	если бы не
43	8	не так ли
44	8	на то чтобы
45	8	по направлению к
46	8	не в силах
47	8	как раз в
48	7	в отличие от
49	7	у него было
50	7	тем более что
51	7	что они не
52	7

In [38]:
# save the output
infile_name = "hp1-1"
out_dir = "/home/philipp/projects/text-analysis/"

removed_stop = "rms" if remove_stopwords else ""
lemmatized = "lm" if lemmatize else ""
removed_other = "".join(filter_pos)
outfile_name = f"{infile_name}_{n}gram_{top_ngrams}_{removed_stop}{lemmatized}_{removed_other}"
out_path = f'{out_dir}{outfile_name}.txt'
with open(out_path, 'w') as the_file:
    the_file.write(out_str)
print(f"Saved to {out_path}.")

Saved to /home/philipp/projects/text-analysis/hp1-1_1gram_1000__NUMPUNCTPROPN.txt.
